# Read HCUP Data


In this notebook, we will demonstrate how to read Healthcare Cost and Utilization Project ([HCUP](https://www.hcup-us.ahrq.gov/)) State Inpatient Data ([SID](https://www.hcup-us.ahrq.gov/sidoverview.jsp)) data for analysis and modelling. The HCUP is the largest collection of longitudinal hospital care data in the United States. The data can be purchased from [here](https://www.hcup-us.ahrq.gov/tech_assist/centdist.jsp).

The HCUP SID dataset consists of four files in ASCII format:
- Core data, including diagnoses, procedures, and patient demographics [[sample](https://github.com/Azure/cortana-intelligence-population-health-management/blob/master/Azure%20Data%20Lake/ManualDeploymentGuide/Model/SampleHCUPdata/Sample_WA_SID_2011_CORE.asc) | [full description](https://www.hcup-us.ahrq.gov/db/state/sidc/tools/filespecs/WA_SID_2011_CORE.loc)]
- Charges associated with each inpatient visit [[sample](https://github.com/Azure/cortana-intelligence-population-health-management/blob/master/Azure%20Data%20Lake/ManualDeploymentGuide/Model/SampleHCUPdata/Sample_WA_SID_2011_CHGS.asc) | [full description](https://www.hcup-us.ahrq.gov/db/state/sidc/tools/filespecs/WA_SID_2011_CHGS.loc)]
- Severity of pre-existing conditions that may affect outcomes [[sample](https://github.com/Azure/cortana-intelligence-population-health-management/blob/master/Azure%20Data%20Lake/ManualDeploymentGuide/Model/SampleHCUPdata/Sample_WA_SID_2011_SEVERITY.asc) | [full description](https://www.hcup-us.ahrq.gov/db/state/sidc/tools/filespecs/WA_SID_2011_SEVERITY.loc)]  
- Diagnostic and procedure groups [[sample](https://github.com/Azure/cortana-intelligence-population-health-management/blob/master/Azure%20Data%20Lake/ManualDeploymentGuide/Model/SampleHCUPdata/Sample_WA_SID_2011_DX_PR_GRPS.asc) | [full description](https://www.hcup-us.ahrq.gov/db/state/sidc/tools/filespecs/WA_SID_2011_DX_PR_GRPS.loc)] 

We will use these description files to help convert the sample ASCII data files to CSV files with headers, which we will then use for subsequent analysis. We include some description of the process in annotated code for the first file, as well as unannotated code for processing the remaining files and combining them into a single CSV file.

In [1]:
# clear workspace and collect garbage
rm(list=ls())
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,440515,23.6,750400,40.1,592000,31.7
Vcells,812815,6.3,1650153,12.6,965682,7.4


## Annotated example: parsing the core dataset

We first download the dataset description file, which contains 20 lines of header information followed by lines containing data in fixed-width columns. We define the beginning and end positions of each fixed-width column in `str_start` and `str_end`, then use `read.fwf()` to parse the fixed-width file into a dataframe.

In [2]:
file <- 'core_desc.txt'
download.file('https://www.hcup-us.ahrq.gov/db/state/sidc/tools/filespecs/WA_SID_2011_CORE.loc', file)

str_start <- c(1, 5, 10, 27, 31, 58, 63, 68, 70, 75)
str_end <- c(3, 8, 25, 29, 56, 61, 66, 68, 73, 174)
widths <- (str_end - str_start) + 2

des <- read.fwf(file, skip=20, widths=widths)
head(des)

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10
WA,2011,CORE,1,AGE,1,3,NA,Num,Age in years at admission
WA,2011,CORE,2,AGEDAY,4,6,NA,Num,Age in days (when age < 1 year)
WA,2011,CORE,3,AGEMONTH,7,9,NA,Num,Age in months (when age < 11 years)
WA,2011,CORE,4,AHOUR,10,13,NA,Num,Admission Hour
WA,2011,CORE,5,AMONTH,14,15,NA,Num,Admission month
WA,2011,CORE,6,ATYPE,16,17,NA,Num,Admission type


From this list, we extract the column names, lightly-reformatted column descriptions, and the beginning and end positions of each column in the fixed-width data file:

In [3]:
des$V10 <- gsub(',', '_', des$V10)
colnameslong <- data.frame(core_2011=des$V10)
colnames <- data.frame(core_2011=des$V5)

str_start <- des$V6
str_end <- des$V7
widths <- (str_end - str_start) + 1

Now we are ready to download and parse the core data file, saving the output in CSV format:

In [4]:
download.file('https://phm.blob.core.windows.net/models/Sample_WA_SID_2011_CORE.asc',
              'Sample_WA_SID_2011_CORE.asc')
dat_core <- read.fwf('Sample_WA_SID_2011_CORE.asc', widths=widths)
dim(dat_core)
names(dat_core) <- trimws(colnames$core_2011)
head(dat_core)
# write.csv(dat_core, 'Sample_WA_SID_2011_CORE.csv', row.names=F)

[1]  10 259

AGE,AGEDAY,AGEMONTH,AHOUR,AMONTH,ATYPE,AWEEKEND,DHOUR,DIED,DISPUB04,...,TOTCHG,TOTCHG_X,TRAN_IN,TRAN_OUT,VisitLink,YEAR,ZIP3,ZIPINC_QRTL,ZIP,AYEAR
52,-99,-99,700,1,3,0,1400,0,1,...,56511,56510.96,0,0,36389,2011,981,2,98122,2011
65,-99,-99,1000,1,3,0,1400,0,6,...,140956,140956.47,0,0,36390,2011,981,3,98144,2011
87,-99,-99,700,1,3,0,1100,0,1,...,12687,12687.35,0,0,36391,2011,981,3,98109,2011
23,-99,-99,500,1,3,0,1400,0,1,...,23402,23402.20,0,0,36392,2011,980,4,98033,2011
30,-99,-99,2200,11,3,0,1300,0,1,...,240352,240351.70,0,0,36393,2011,982,4,98208,2010
29,-99,-99,1200,11,3,0,600,0,5,...,282202,282202.20,0,1,36394,2011,982,1,98225,2010


## Unannotated code for the remaining data

### Charges associated with each inpatient visit

In [5]:
file <- 'charges_desc.txt'
download.file('https://www.hcup-us.ahrq.gov/db/state/sidc/tools/filespecs/WA_SID_2011_CHGS.loc', file)

str_start <- c(1, 5, 10, 27, 31, 58, 63, 68, 70, 75)
str_end <- c(3, 8, 25, 29, 56, 61, 66, 68, 73, 174)
widths <- (str_end - str_start) + 2

des <- read.fwf(file, skip=20, widths=widths)

des$V10 <- gsub(',', '_', des$V10)
colnameslong <- data.frame(charges_2011=des$V10)
colnames <- data.frame(charges_2011=des$V5)

str_start <- des$V6
str_end <- des$V7
widths <- (str_end - str_start) + 1

download.file('https://phm.blob.core.windows.net/models/Sample_WA_SID_2011_CHGS.asc',
              'Sample_WA_SID_2011_CHGS.asc')
dat_chrg <- read.fwf('Sample_WA_SID_2011_CHGS.asc', widths=widths)
dim(dat_chrg)
names(dat_chrg) <- trimws(colnames$charges_2011)
head(dat_chrg)
# write.csv(dat_chrg, 'Sample_WA_SID_2011_CHGS.csv', row.names=F)

[1]  10 158

KEY,NREVCD,REVCD1,REVCD2,REVCD3,REVCD4,REVCD5,REVCD6,REVCD7,REVCD8,...,UNIT43,UNIT44,UNIT45,UNIT46,UNIT47,UNIT48,UNIT49,UNIT50,UNIT51,UNIT52
5.320111e+14,15,1,120,250,258,272,278,300,305,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999
5.320111e+14,18,1,120,250,258,270,272,278,300,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999
5.320111e+14,15,1,120,250,270,272,300,301,305,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999
5.320111e+14,11,1,122,250,270,272,300,305,360,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999
5.320111e+14,13,1,129,250,300,301,302,305,306,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999
5.320111e+14,12,1,129,250,300,301,302,305,306,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999


### Severity of pre-existing conditions that may affect outcomes

In [6]:
file <- 'severity_desc.txt'
download.file('https://www.hcup-us.ahrq.gov/db/state/sidc/tools/filespecs/WA_SID_2011_SEVERITY.loc', file)

str_start <- c(1, 5, 10, 27, 31, 58, 63, 68, 70, 75)
str_end <- c(3, 8, 25, 29, 56, 61, 66, 68, 73, 174)
widths <- (str_end - str_start) + 2

des <- read.fwf(file, skip=20, widths=widths)

des$V10 <- gsub(',', '_', des$V10)
colnameslong <- data.frame(severity_2011=des$V10)
colnames <- data.frame(severity_2011=des$V5)

str_start <- des$V6
str_end <- des$V7
widths <- (str_end - str_start)+1

download.file('https://phm.blob.core.windows.net/models/Sample_WA_SID_2011_SEVERITY.asc',
              'Sample_WA_SID_2011_SEVERITY.asc')
dat_sevr <- read.fwf('Sample_WA_SID_2011_SEVERITY.asc', widths=widths)
dim(dat_sevr)
names(dat_sevr) <- trimws(colnames$severity_2011)
head(dat_sevr)
# write.csv(dat_sevr, 'Sample_WA_SID_2011_SEVERITY.csv', row.names = F )

[1] 10 30

KEY,CM_AIDS,CM_ALCOHOL,CM_ANEMDEF,CM_ARTH,CM_BLDLOSS,CM_CHF,CM_CHRNLUNG,CM_COAG,CM_DEPRESS,...,CM_OBESE,CM_PARA,CM_PERIVASC,CM_PSYCH,CM_PULMCIRC,CM_RENLFAIL,CM_TUMOR,CM_ULCER,CM_VALVE,CM_WGHTLOSS
5.320111e+14,1,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5.320111e+14,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
5.320111e+14,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5.320111e+14,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5.320111e+14,0,1,1,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5.320111e+14,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0


### Diagnostic and procedure groups

In [7]:
file <- 'dxpr_desc.txt'
download.file('https://www.hcup-us.ahrq.gov/db/state/sidc/tools/filespecs/WA_SID_2011_DX_PR_GRPS.loc', file)

str_start <- c(1, 5, 10, 27, 31, 58, 63, 68, 70, 75)
str_end <- c(3, 8, 25, 29, 56, 61, 66, 68, 73, 174)
widths <- (str_end - str_start) + 2

des <- read.fwf(file, skip=20, widths=widths)

des$V10 <- gsub(',', '_', des$V10)
colnameslong <- data.frame(dxpr_2011=des$V10)
colnames <- data.frame(dxpr_2011=des$V5)

str_start <- des$V6
str_end <- des$V7
widths <- (str_end - str_start) + 1

download.file('https://phm.blob.core.windows.net/models/Sample_WA_SID_2011_DX_PR_GRPS.asc',
              'Sample_WA_SID_2011_DX_PR_GRPS.asc')
dat_dxpr <- read.fwf('Sample_WA_SID_2011_DX_PR_GRPS.asc', widths=widths)
dim(dat_dxpr)
names(dat_dxpr) <- trimws(colnames$dxpr_2011)
head(dat_dxpr)
# write.csv(dat_dxpr, 'Sample_WA_SID_2011_DX_PR_GRPS.csv', row.names=F)

[1]  10 182

CHRON1,CHRON2,CHRON3,CHRON4,CHRON5,CHRON6,CHRON7,CHRON8,CHRON9,CHRON10,...,U_OCCTHERAPY,U_ORGANACQ,U_OTHIMPLANTS,U_PACEMAKER,U_PHYTHERAPY,U_RADTHERAPY,U_RESPTHERAPY,U_SPEECHTHERAPY,U_STRESS,U_ULTRASOUND
1,1,1,1,1,1,1,1,1,1,...,1,0,1,0,1,0,0,0,0,0
0,0,0,1,1,1,1,1,1,1,...,1,0,1,0,1,0,0,0,0,0
0,1,1,0,1,1,1,1,1,0,...,1,0,0,0,1,0,0,0,0,0
0,0,1,0,0,0,-9,-9,-9,-9,...,0,0,0,0,0,0,0,0,0,0
1,1,0,1,1,1,1,0,0,1,...,0,0,0,0,1,0,0,0,0,1
1,1,1,0,0,1,1,1,1,1,...,0,0,0,0,0,0,1,0,0,1


## Merge results into a single CSV-formatted dataset

In [8]:
dat <- merge(merge(merge(dat_sevr, dat_chrg, by="KEY"), dat_core, by="KEY"), dat_dxpr, by="KEY")  
dim(dat)
head(dat)
write.csv(dat, 'Sample_WA_SID_2011.csv', row.names=F)

[1]  10 626

KEY,CM_AIDS,CM_ALCOHOL,CM_ANEMDEF,CM_ARTH,CM_BLDLOSS,CM_CHF,CM_CHRNLUNG,CM_COAG,CM_DEPRESS,...,U_OCCTHERAPY,U_ORGANACQ,U_OTHIMPLANTS,U_PACEMAKER,U_PHYTHERAPY,U_RADTHERAPY,U_RESPTHERAPY,U_SPEECHTHERAPY,U_STRESS,U_ULTRASOUND
5.320111e+14,1,0,0,1,0,0,0,0,1,...,1,0,1,0,1,0,0,0,0,0
5.320111e+14,0,0,0,0,0,0,1,0,0,...,1,0,1,0,1,0,0,0,0,0
5.320111e+14,0,0,0,0,0,1,0,0,0,...,1,0,0,0,1,0,0,0,0,0
5.320111e+14,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5.320111e+14,0,1,1,0,1,0,0,0,1,...,0,0,0,0,1,0,0,0,0,1
5.320111e+14,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,1
